In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [3]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [4]:
df_q1_agg = df_q1.groupby("prod")["purchase"].sum()
df_q1_agg[df_q1_agg == df_q1_agg.max()]

prod
P00025442    27995166
Name: purchase, dtype: int64

In [5]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [6]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [7]:
df_q1_sub["job"].value_counts().idxmax()

4

### Q2.

In [8]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].reset_index(drop = True)
df_u1 = df_u1.fillna(0)

In [9]:
df_u1.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0


In [10]:
# df_u1.sort_values(["prod_cat1", "prod_cat2", "prod_cat3"])

In [11]:
df_u1.drop_duplicates().shape

(21, 3)

In [12]:
# .astype("int") 는 이쁘게 하기 위해.... 안해도 됨.
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("int").astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat" ] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [13]:
df_u1.head(1)

,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,0,3-0-0


In [14]:
df_u1["prod_cat"].nunique()

21

In [15]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [16]:
df_q2 = df.loc[df["age_group"] == "26-35", ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
df_q2 = df_q2.fillna(0)

In [17]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [18]:
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0


In [19]:
# df_q2.groupby("user")["prod_cat"].nunique()
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_agg.head(2)

,user,marital,prod_cat
0,3,0,18
1,5,1,43


In [20]:
stat_m0 = df_q2_agg.loc[df_q2_agg["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_agg.loc[df_q2_agg["marital"] == 1, "prod_cat"].mean()
stat_m0, stat_m1

(41.66318327974277, 41.79233621755253)

In [21]:
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

고객별로 중복된 정보가 있는 경우.

In [22]:
df_s = df[["user", "gender", "age_group", "job", "city", "marital"]].sample(3, random_state = 1)
df_s = pd.concat([df_s, df_s]).reset_index(drop = True)
df_s.iloc[0, 4] = "D"
df_s

,user,gender,age_group,job,city,marital
0,413,M,26-35,11,D,0
1,984,M,51-55,16,A,1
2,2106,F,18-25,20,A,1
3,413,M,26-35,11,C,0
4,984,M,51-55,16,A,1
5,2106,F,18-25,20,A,1


In [23]:
df_s_cnt = df_s.groupby("user")["city"].nunique()
df_s_cnt[df_s_cnt > 1].index.to_list() # 이사를 한 것으로 추정되는 고객정보 뽑기

[413]

In [24]:
df_user = df[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()
df_user = df_user.reset_index(drop = True)

In [25]:
df_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0


In [26]:
df["user"].nunique(), len(df_user)

(5891, 5891)

In [27]:
df_q3_agg = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique", 
                                                          "purchase": "sum"})
df_q3_agg = df_q3_agg.reset_index()
df_q3_agg.head(3)

,user,prod,purchase
0,1,35,334093
1,2,77,810472
2,3,29,341635


In [28]:
df_q3 = df_user.merge(df_q3_agg, on = "user", how = "inner")
df_q3 = df_q3.drop(columns = "user")
df_q3.head(2)

,gender,age_group,job,city,marital,prod,purchase
0,F,0-17,10,A,0,35,334093
1,M,55+,16,C,0,77,810472


In [29]:
df_q3["gender"] = df_q3["gender"].replace({"M": 1, "F": 0})

In [30]:
arr_u = df_q3["age_group"].unique()
# arr_u = arr_u.sort() # 이렇게 하면 안됨.
arr_u.sort()
arr_u

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [31]:
ser_repl = pd.Series(range(7), index = arr_u)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [32]:
pd.crosstab(df_q3["age_group"], df_q3["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [33]:
df_q3["age_group"] = df_q3["age_group"].replace(ser_repl)

In [34]:
# df_q3_dum = pd.get_dummies(df_q3, columns = ["job", "city"]) # 시험버전
df_q3_dum = pd.get_dummies(df_q3, columns = ["job", "city"], dtype = "int")
df_q3_dum.head(1)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [35]:
arr_q3_nor = MinMaxScaler().fit_transform(df_q3_dum)
arr_q3_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [36]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(arr_q3_nor)

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7, random_state=123)

In [37]:
round(silhouette_score(arr_q3_nor, labels = model_kmeans.labels_), 2)

0.18

In [38]:
df_q3_dum["cluster"] = model_kmeans.labels_

In [39]:
df_q3_dum["cluster"].value_counts() # 각 군집의 크기

cluster
1    1291
0    1004
5     980
2     868
6     703
3     652
4     393
Name: count, dtype: int64